In [39]:
# The display function isn't always imported by default in some Jupyter implementations.
# We'll probably use it a lot.
from IPython.core.display import display, HTML

In [135]:
# Jupyter can output HTML if we want. Here's a simple helper to make headings
# and other embellishments easy:
def html_wrap(content, element="span", attributes=None):
    """ Convenience method for wrapping any string in an HTML tag. """
    element_with_attributes = ' '.join((item for item in (element, attributes) if item))
    tag_open = f"<{element_with_attributes}>"
    tag_close = f"</{element}>"
    return f"{tag_open}{content}{tag_close}"

def _heading(content, heading_level=5):
    """ Convenience method for wrapping any string in an HTML header tag. """
    element = f"h{heading_level}"
    return html_wrap(content, element)

def heading(*args, **kwargs):
    return HTML(_heading(*args, **kwargs))

In [40]:
# pandas is really good with columnar data, like Excel files - https://pandas.pydata.org
import pandas

In [41]:
# The file I'm interested in parsing for cleanup:
file_path = "./src/real data/2017 CAM data from iPads/2017 CAM data from iPads.xlsx"

In [42]:
data_file = pandas.ExcelFile(file_path)
sorted(data_file.sheet_names)

['2017 CAM data Erl',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data',
 'schema (WIP reverse engineer)']

In [43]:
# I'm only interested in first two, for now:
cam_sheet_names = _[:2]

In [44]:
# make a dictionary of dataframes from all the sheets, using the last word as the name (index -1 means last)
sheets = {sheet_name.split(' ')[-1]: data_file.parse(sheet_name)
          for sheet_name in cam_sheet_names}

In [45]:
# The keys are sheet names. Let's see what we've got:
list(sheets.keys())

['Erl', 'Tyler']

Now that we have a convenient list of sheets that are loaded as _pandas_ DataFrames, we can work toward merging them into one. Once they're merged, we can process the data more easily from a single DataFrame.

## Preparing to merge: method

_pandas_ has a [`concat`] method for tacking one dataframe onto another. It also has [`append`] and [`merge`]. Time to decide which will serve us best.

[`concat`]: https://pandas.pydata.org/pandas-docs/stable/merging.html
[`append`]: https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-using-append
[`merge`]: https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

The concatenate method has an option to ignore row numbers, which has the effect of gluing each dataframe to the bottom of the previous one. Append would be almost equivalent, but always updates the first DataFrame or Series it's given. Merge is more for database-style relational merges.

From reading up on all three methods, it looks to me like `concat` will suffice, here, as long as the column names are identical. I like to have the option of creating a new DataFrame rather than overwriting anything, because the results are easier to repeat.

Let's see how close that concatenation method gets us:

In [46]:
pandas.concat(sheets, ignore_index=True).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4690 entries, 0 to 4689
Data columns (total 56 columns):
clients__company                                                    1 non-null object
clients__displayText                                                1 non-null object
clients__fname                                                      1 non-null object
clients__lname                                                      1 non-null object
clients__name                                                       1 non-null object
fields__client__company                                             21 non-null object
fields__client__displayText                                         21 non-null object
fields__client__fname                                               21 non-null object
fields__client__lname                                               21 non-null object
fields__client__name                                                21 non-null object
fields__crop                

We can see the RangeIndex size in the info: 

>4690 entries, 0 to 4689

Which means 4690 rows if this were a spreadsheet. Compared to the total row count for all sheets:

In [47]:
sum(len(sheet) for sheet in sheets.values())

4690

Pretty good, so far, except for some variations in the column names—I see "growthStage" and "growthStage Zadoks", as well as some aphid names amongst the "a1__number", "a2__number", "a3__number" group. 

```
fields__oSets__growthStage                                  45 non-null float64
fields__oSets__growthStage Zadoks                           9 non-null float64
```

```
fields__oSets__oPoints__observations__a1__number            324 non-null float64
fields__oSets__oPoints__observations__a1__number EGA        124 non-null float64
fields__oSets__oPoints__observations__a2__number            112 non-null float64
fields__oSets__oPoints__observations__a2__number BCO        30 non-null float64
fields__oSets__oPoints__observations__a3__number            25 non-null float64
fields__oSets__oPoints__observations__a3__number Greenbug   0 non-null float64
```

The total number of rows is correct, and the other columns line up when the names match.

Column names are the only problem standing in the way of a successful unification of DataFrames.

## Preparing to merge: column names

Quick comparison of column names, showing differences:

In [48]:
display(sheets.keys(),
        set.difference(*[set(sheet.columns) for sheet in sheets.values()]))

dict_keys(['Erl', 'Tyler'])

{'fields__oSets__growthStage Zadoks',
 'fields__oSets__oPoints__observations__a1__number EGA',
 'fields__oSets__oPoints__observations__a2__number BCO',
 'fields__oSets__oPoints__observations__a3__number Greenbug',
 'fields__oSets__oPoints__observations__anum TotalAPhids',
 'fields__oSets__oPoints__observations__eVnum Natural enemy totals'}

In [49]:
# This time we want columns from *all* sheets that don't have a match
# (symmetrical difference).
column_list = sorted(set.symmetric_difference(*[set(sheet.columns) for sheet in sheets.values()]))

In [50]:
# Display entire column, even if cell data is long
pandas.set_option('display.max_colwidth', 0)

# Show the differing column names side-by-side
# (sorry about the magical gibberish)
pandas.DataFrame([*zip(*[iter(column_list)] * 2)])

,0,1
0,fields__oSets__growthStage,fields__oSets__growthStage Zadoks
1,fields__oSets__oPoints__observations__a1__number,fields__oSets__oPoints__observations__a1__number EGA
2,fields__oSets__oPoints__observations__a2__number,fields__oSets__oPoints__observations__a2__number BCO
3,fields__oSets__oPoints__observations__a3__number,fields__oSets__oPoints__observations__a3__number Greenbug
4,fields__oSets__oPoints__observations__anum,fields__oSets__oPoints__observations__anum TotalAPhids
5,fields__oSets__oPoints__observations__eVnum,fields__oSets__oPoints__observations__eVnum Natural enemy totals


When we concatenate, we'll use the names from the first sheet, since it's more regular. Which sheet is the one to fix? Which sheet has '`fields__oSets__growthStage Zadoks`' instead of '`fields__oSets__growthStage`'?

In [136]:
# make a filtered dictionary, for later reference
sheets_with_bad_column_names = {sheet_name: sheet for sheet_name, sheet in sheets.items() if 'fields__oSets__growthStage Zadoks' in sheet.columns}
display(heading('Bad:'),
        set(sheets_with_bad_column_names.keys()))

set()

In [137]:
display(heading('Good:'),
        set(sheets.keys()) - set(sheets_with_bad_column_names.keys()))

{'Erl', 'Tyler'}

Okay, "Tyler" has the column names we prefer, "Erl" does not. Duly noted. There's now a `sheets_with_bad_column_names` variable we can use to fix that. (It's only got one sheet, but it's still good practice to be ready for batch processing in case this code is reused in the future.)

How shall we solve the name mismatch? Our options:

- rename the columns
- try to merge the sheets in a way that ignores the column names of the bad sheet

If we try to do a special merge, ignoring the column names, we have to worry about the precise order of those columns instead of relying on the names. Since we'd have to spend time checking—and possibly rearranging—the column sequence, we might as well spend that time creating a _reusable_, documented solution for quick and painless name fixing.

How (where) should we rename the columns? Possibilities:

- fix the spreadsheet document in Excel format, and move ahead as if this never happened
- keep the names in the file, but correct the names in memory, using automation

Since this document has already mentioned the problem, let's go ahead and solve it here. That way our notebook will detail the fix and how to use it in the future, if there are more Excel spreadsheets with similarly altered names.

### Renaming columns

_pandas_ has a [`rename`] method which lets us apply a transform function to the columns (or explicitly map each column through a dictionary).

[`rename`]: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html

Since the bad names contain suffixes beginning with a space character, let's just ignore anything after and including the first space character in a column name. It's easy to do in Python:

In [53]:
def first_word(column_name, word_separator=' '):
    """ Split string into words (by space character), return first word. """
    return column_name.split(word_separator)[0]  # [0] means first item in list

In [54]:
report = []  # start a list of strings to display after renaming
for sheet_name, sheet in sheets_with_bad_column_names.items():
    report.append(f"<h2>{sheet_name}</h2>")  # heading
    report.append('<ol>')  # start an ordered list
    for column_name in sheet.columns:
        after = first_word(column_name)
        boldness = 'bold' if after != column_name else 'normal'
        report.append(f"<li style='font-weight: {boldness}'>{column_name} &rarr; {after}</li>")
    report.append('</ol>')  # close the list
    sheet.rename(mapper=first_word, axis='columns', inplace=True)
display(HTML(''.join(report)))  # join all the strings together and display as HTML

## Merge all sheets together

In [55]:
# @todo: replace experimental_concat with a good one
# @todo: replace examples and documented references to multiple sheets with the unified one

## Making sense of really long, repetitive column names

_pandas_ has a concept of indexing hierarchically, which may help group columns together. However, I'm new to _pandas_ and I'm not new to Python. Furthermore, from what I've read, if we use a multi-level labelling system for grouping, we have to pay attention to which level we're operating on as we work with the spreadsheet as DataFrame. To avoid complications, let's use a nested dictionary in Python to accomplish the same thing without _pandas_.

### Python dictionaries

Dictionaries in Python are just collection of named things. The things can be another dictionary, a string, a number, or whatver.

In [56]:
# define a dictionary:
my_dictionary = {
    'one': 1,
    'green': 'I like colour green.',
    'another dictionary': {'more stuff': 1024,
                           'even more stuff': 2048},
    'a list': [1, 2, 3, 4]
}

In [57]:
# recall something from that dictionary:
my_dictionary['one']

1

In [58]:
my_dictionary['another dictionary']['even more stuff']

2048

### Nested dictionaries, for a hierarchy

Instead of a regular dictionary in Python, let's use something called a [`defaultdict`], which will simplify the creation of our hierarchy. [`defaultdict`] is like a regular dictionary, except it doesn't complain if you try to access a key that doesn't exist yet—it just adds it.

[`defaultdict`]: https://docs.python.org/3/library/collections.html

The following method looks long but it's mostly comments for your benefit. 

In [138]:
from collections import defaultdict

def split_column_to_dict(sheet, column_name, column_dictionary=None, separator='__'):
    """ Split the column names like "fields__oSets__oPoints__observations" into groupings of keys
    so that related keys are easy to find, ie columns['fields']['oSets']['oPoints']['observations'].
    This produces a tree of column name segments, with actual pandas data at the ends."""

    # If a dictionary is not provided, make an empty one.
    if column_dictionary is None:
        def nested_dict():
            """ This function will be called by defaultdict
            whenever a non-existent key is used. """
            return defaultdict(nested_dict)
        
        column_dictionary = nested_dict()

    # Set a pointer to the root of the tree, for starters.
    pointer = column_dictionary
    
    # Now, walk through the segments in order from left to right,
    # touching the tree for each one.
    for segment in str(column_name).split(separator):
        # Just update the pointer to the deeper location in the tree.
        pointer = pointer[segment]
    # Now that the loop is done, the pointer is pointing at the deepest
    # level of the branch, which either already existed or it was created.
    
    # At the end of the branch, put the data.
    # NOTE: To avoid naming conflicts with pandas magic attributes
    # (such as "number"), the reference to data is in a node with a name that
    # can't possibly be part of naming levels: the separator ('__') 
    pointer[separator] = sheet[column_name]  # the actual pandas "column"

    # Since `pointer` was actually just pointing to parts of the column dictionary,
    # the column dictionary has been filled out with nodes because of how defaultdict
    # was setup with our `nested_dict` constructor.
    return column_dictionary


In [60]:
# Now, make a dictionary of column trees, grouped by sheet name.
column_dictionary = {}
separator = '__'
for sheet_name, sheet in sheets.items():
    # Build a new nested column dictionary for this sheet
    new_dict = None
    for column in sheet.columns:
        new_dict = split_column_to_dict(sheet, column, new_dict, separator)
    # Turn off the defaultdict behaviour of creating a key instead of throwinng an exception
    new_dict.default_factory = None
    column_dictionary[sheet_name] = new_dict


If this worked, there should now be a list of sheets at the top:

In [61]:
column_dictionary.keys()

dict_keys(['Erl', 'Tyler'])

Then there should be a list of the first segments of all the column names in that sheet:

In [62]:
column_dictionary['Erl'].keys()

dict_keys(['fields', 'clients', 'observers'])

Continuing deeper, more segments that share a common prefix:

In [63]:
column_dictionary['Erl']['fields'].keys()

dict_keys(['client', 'name', 'crop', 'desc', 'image', 'date', 'oSets'])

In [64]:
column_dictionary['Erl']['fields']['oSets'].keys()

dict_keys(['date', 'dateCompare', 'growthStage', 'desc', 'obsName', 'totalSets', 'completeSets', 'results', 'oPoints', 'totalA1', 'totalA2', 'totalA3', 'totalA4'])

At the end of each branch in the tree should be a '\_\_' key for the actual data. For example, the `date` of the set:

In [65]:
column_dictionary['Erl']['fields']['oSets']['date'].keys()

dict_keys(['__'])

Furthermore, if we want to skip over blanks as we scan down a column, _pandas_ has a [`dropna()`] method for that:

[`dropna()`]: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html

In [66]:
column_dictionary['Erl']['fields']['oSets']['date'][separator].dropna()

0      2017-08-02T13:12:09.542
70     2017-08-09T09:25:11.710
140    2017-08-09T10:06:25.480
210    2017-08-09T11:21:01.555
350    2017-08-09T11:37:20.862
490    2017-08-22T15:42:05.751
560    2017-08-17T11:12:02.820
700    2017-08-17T13:06:30.183
840    2017-08-22T16:02:50.682
Name: fields__oSets__date, dtype: object

We now have enough information to find the beginning of chunks in the sheet by scanning for non-blank cells.

As a side note, the number of non-null (not empty) records in any given column was displayed when we called [`info()`] on the DataFrame. We can also get information like that from the column (which is a Series in _pandas_) by using [`describe()`] or [`count()`]:

[`info()`]: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html
[`describe()`]: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.describe.html
[`count()`]: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.count.html

In [81]:
column_dictionary['Erl']['fields']['oSets']['date'][separator].describe()

count     9                      
unique    9                      
top       2017-08-09T11:37:20.862
freq      1                      
Name: fields__oSets__date, dtype: object

### Helper functions, for browsing the data

Once we start actually reading groups of columns based on which type of chunk we're reading from, we'll have to start typing lists of column names for each section.

It's a bit of extra work to pay attention to whether a given node in the hierarchy is holding a reference to a data Series (column) or whether it's only an intermediate step on the way to the end of the branch.

To save time and mental energy, we can filter columns by whether they have data or not, if we make some simple filter functions. For detecting data that we've placed in the hierarchy of columns, we can look for the special key that we chose earlier (the separator, which is two underscore characters `__`).

In [86]:
def has_data(node):
    """ Filter children that have data. We know a child item has data
    if it has a key that's the just the separator string. """
    return {parent_key: child for parent_key, child in node.items()
            if separator in child.keys()}


def has_children(node):
    """ Filter children that have children. We know an item has children
    if it has at least one key that isn't just the separator string, the
    special key for data references. """
    return {parent_key: child for parent_key, child in node.items()
            if len([key for key in child.keys() if key != separator]) >= 1}

In [139]:
example_node = column_dictionary[sheet_name]['fields']['oSets']['oPoints']['observations']
display(heading('children with children:'),
        has_children(example_node).keys())

dict_keys(['a1', 'a2', 'a3', '|'])

In [140]:
display(heading('children with data:'),
        has_data(example_node).keys())

dict_keys(['id', 'name', 'enum', 'eVnum', 'anum', 'disabled', 'complete', '|'])

In [141]:
# Set of keys for nodes that that have children but also data:
display(heading('children with children & data:'),
        set(has_children(example_node).keys()) & set(has_data(example_node).keys()))

{'|'}

#### Personal thought:

I have to wonder why the developers of the app that output this data chose to use an unpronounceable column name, and put something important there.

Regardless, we can effortlessly handle it now.

## Repeated elements: fields, sets, points, observations

Let's take a look at the columns containing data about observation sets:

In [142]:
for sheet_name, column_tree in column_dictionary.items():
    node = column_tree['fields']['oSets']
    date_column = node['date'][separator]
    columns = [child[separator].name for parent_key, child in has_data(node).items()]
    display(heading(f"{sheet_name}:"),
            sheets[sheet_name].loc[date_column.isna() != True, columns].head(3))

,fields__oSets__date,fields__oSets__dateCompare,fields__oSets__growthStage,fields__oSets__desc,fields__oSets__obsName,fields__oSets__totalSets,fields__oSets__completeSets,fields__oSets__results,fields__oSets__totalA1,fields__oSets__totalA2,fields__oSets__totalA3,fields__oSets__totalA4
0,2017-08-02T13:12:09.542,2017-08-02,7.0,NaN,Tyler,1.0,0.0,NaN,NaN,NaN,NaN,NaN
70,2017-08-09T09:25:11.710,2017-08-09,8.0,NaN,Tyler,1.0,1.0,RESULTS.5,164.0,0.0,0.0,0.0
140,2017-08-09T10:06:25.480,2017-08-09,7.0,NaN,Tyler,1.0,1.0,RESULTS.5,66.0,0.0,0.0,0.0


,fields__oSets__date,fields__oSets__dateCompare,fields__oSets__growthStage,fields__oSets__desc,fields__oSets__obsName,fields__oSets__totalSets,fields__oSets__completeSets,fields__oSets__results,fields__oSets__totalA1,fields__oSets__totalA2,fields__oSets__totalA3,fields__oSets__totalA4
0,2017-07-14T12:31:24.194,2017-07-14,6.0,NaN,Tyler,1.0,0.0,NaN,NaN,NaN,NaN,NaN
70,2017-07-18T10:31:22.263,2017-07-18,6.0,NaN,Tyler,1.0,1.0,RESULTS.5,8.0,0.0,0.0,0.0
140,2017-07-28T13:05:44.673,2017-07-28,8.0,NaN,Mikki,1.0,1.0,RESULTS.5,37.0,0.0,0.0,0.0


We can ask _pandas_ to drop all the blanks and see which rows remain for the column here (`fields__oSets__date`):

In [143]:
column_dictionary['Erl']['fields']['oSets']['date'][separator].dropna().index

Int64Index([0, 70, 140, 210, 350, 490, 560, 700, 840], dtype='int64')

How many 'oSets' sections of the file are there?

In [144]:
len(_)  # Based on the length of that list we just output

9

Good to know. What about the other file?

In [145]:
column_dictionary['Tyler']['fields']['oSets']['date'][separator].count()

45

In [74]:
_ + __  # add the last two numbers

54

So, 54 sets of observations we'll be processing.

How many observation _points_ will we be processing?

In [75]:
pointer = column_dictionary['Tyler']['fields']['oSets']
has_children(pointer)

{'oPoints': defaultdict(<function __main__.split_column_to_dict.<locals>.nested_dict>,
             {'id': defaultdict(<function __main__.split_column_to_dict.<locals>.nested_dict>,
                          {'__': 0       0.0
                           1      NaN 
                           2      NaN 
                           3      NaN 
                           4      NaN 
                           5      NaN 
                           6      NaN 
                           7      NaN 
                           8      NaN 
                           9      NaN 
                           10     NaN 
                           11     NaN 
                           12     NaN 
                           13     NaN 
                           14      1.0
                           15     NaN 
                           16     NaN 
                           17     NaN 
                           18     NaN 
                           19     NaN 
                           20   